In [14]:
%matplotlib inline
import numpy as np

import matplotlib.pyplot as plt

from IPython.display import set_matplotlib_formats
set_matplotlib_formats('pdf', 'png')
plt.rcParams['savefig.dpi'] = 75

plt.rcParams['figure.autolayout'] = False
plt.rcParams['figure.figsize'] = 10, 6
plt.rcParams['axes.labelsize'] = 18
plt.rcParams['axes.titlesize'] = 20
plt.rcParams['axes.grid'] = True
plt.rcParams['font.size'] = 16
plt.rcParams['lines.linewidth'] = 2.0
plt.rcParams['lines.markersize'] = 8
plt.rcParams['legend.fontsize'] = 14

plt.rcParams['text.usetex'] = True
plt.rcParams['font.family'] = "serif"
plt.rcParams['font.serif'] = "cm"
plt.rcParams['text.latex.preamble'] = r"\usepackage{subdepth}, \usepackage{type1cm}"

# Project Euler problem 176 - Right-angled triangles that share a cathetus

[Link to problem on Project Euler homepage](https://projecteuler.net/problem=176)

## Description

The four right-angled triangles with sides (9,12,15), (12,16,20), (5,12,13) and (12,35,37) all have one of the shorter sides (catheti) equal to 12. It can be shown that no other integer sided right-angled triangle exists with one of the catheti equal to 12.

Find the smallest integer that can be the length of a cathetus of exactly 47547 different integer sided right-angled triangles.

## Analysis

### Integer solutions to the difference between two square numbers
To solve the problem, we have to find an efficient method of calculating the number of integer solutions to the Pythagorean equation, $a^2 + b^2 = c^2$, with one of the cathetii held constant. One way forward is to solve for one of the cathetii, which yields the difference of two squares, and split that into two factors

$$a^2 = c^2 - b^2 = (c - b)\times(c + b) = p\times q,$$

where

$$
\begin{eqnarray}
p &=& c - b \\
q &=& c + b.
\end{eqnarray}
$$

Now, to find out if there is any kind of restriction on the factors that be written as the difference between two squares let us try to solve for $b$ and $c$

$$
\begin{eqnarray}
b &=& \frac{q - p}{2} \\
c &=& \frac{p + q}{2}.
\end{eqnarray}
$$

For $b$ and $c$ to be integers, $p$ and $q$ need to have the same [parity](https://en.wikipedia.org/wiki/Parity_(mathematics)). Indeed, enumerating all the different ways of splitting $12^2$ into two factors we find

|   $p$   |   $q$   |    $b$   |   $c$    |
|---------|---------|----------|----------|
|     1   |   144   |   71     |   72.5   |
|   **2** |  **72** | **35**   | **37**   |
|     3   |    48   |   22.5   |   25.5   |
|   **4** |  **36** | **16**   | **20**   |
|   **6** |  **24** |  **9**   | **15**   |
|   **8** |  **18** |  **5**   | **13**   |
|     9   |    16   |    3.5   |   12.5   |

The rows in bold are those where $p$ and $q$ have same parity, and corredpond to the four solutions enumerated in the problem description.

### Factorisation of $a^2$

We now know that if $a^2$ can be written as the product between two factors, $p$ and $q$, with the same parity, then it can be written as the difference between two square numbers. The number of solutions is simply the number of unique ways that $a^2$ can be split into two factors with the same parity (we do not count the trivial case with $p=q$ which would yield $b=0$ and $a=c$). By the [Fundamental theorem of arithmetic](https://en.wikipedia.org/wiki/Fundamental_theorem_of_arithmetic) any integer can be written as a unique factorisation of prime numbers, and since $a^2$ is a square number it can be written as

$$a^2 = p_1^{2n_1}p_2^{2n_2} \cdots p_k^{2n_k},$$

where $p_i$ are primes and $2n_i$ the multiplicities.

The number of divisors of a square number is

$$\sigma_0 = \prod_{i=1}^k (2n_i + 1).$$

Generally, the number of ways to split a number into two factors is then $\sigma_0/2$ since each divisor is counted twice. However, since a square number, $a^2$, has an odd number of divisors (due to the fact that $a$ divides $a^2$ twice) the number of ways to split a square number into two factors is $\frac{\sigma_0+1}{2}$.

If all the prime numbers of the factorisation are odd, then both factors will always have odd parity and we are done. The solution is then given as

$$N_\mathrm{split} = \frac{\sigma_0+1}{2} - 1,$$

where the subtraction of 1 is to remove the trivial case with $b=0$ and $a=c$.

However, if $2$ is part of the prime factorisation we have to make sure that $2$ is part of both factors to ensure that both have odd parity. The solution then is

$$N_\mathrm{split} = \frac{(2n_1-1)\prod_{i=2}^k (2n_i + 1) + 1}{2} - 1,$$

where $2n_1$ is the multiplicity of the first prime, $2$. This approach works because there are $n-1$ ways of distributing $n$ indistinguishable items into two groups.

## Algorithm

I am going to assume that $2$ will be part of the prime factorisation of the solution, since we are looking for the smallest square number that can be written as the product of two factors with the same parity in $47547$ different ways.

The simplest approach is to simply look for integer solutions for $N_\mathrm{split} = 47547$, by brute force and then apply the found solutions to the smallest primes such that the smallest prime will have the greatest multiplicity.

I am not going to attempt an analysis of how many factors are needed to find the smallest solution, but will assume that the number of unique prime factors $<10$.

In [119]:
from itertools import count

def n_split(n, depth=1, max_depth=1, previous_product=1, multiplicities=None):
    if depth == 1:
        n = 2*(n + 1) - 1
        multiplicities=[]
    for i in count(1):
        if depth == 1:
            product = 2*i - 1 # first prime - multiplicity is 2*n - 1
        else:
            product = previous_product*(2*i + 1) # subsequent primes - multiplicith is 2*n + 1
        if product > n: # break if number of splits is too high
            break
        if depth < max_depth:
            yield from n_split(n, depth=depth+1,
                                  max_depth=max_depth,
                                  previous_product=product,
                                  multiplicities=multiplicities+[i])
        else:
            if product == n:
                yield product, multiplicities+[i]

In [130]:
from time import time

primes = [2, 3, 5, 7, 11, 13, 17, 19, 23, 29]
nsplit = 47547

t0 = time()

solution = -1
for max_depth in range(1, 11):
    for _, multi in n_split(nsplit, max_depth=max_depth):
        product = 1
        for p, m in zip(primes, multi):
            product *= pow(p, m)
        if solution == -1:
            solution = product
        elif product < solution:
            solution = product

print("Solution = {}".format(solution))
print("Time elapsed = {:.2f}s".format(time()-t0))

96818198400000
Time elapsed = 2.62s
